In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import json
import gensim.downloader as api
from sklearn.metrics import f1_score
import numpy as np
from gensim.models import KeyedVectors
import pickle
import matplotlib.pyplot as plt

In [4]:
with open('Json Task1/NER_train.json', 'r') as f:
    task1_train_data = json.load(f)

with open('Json Task1/NER_val.json', 'r') as f:
    task1_val_data = json.load(f)

with open('Json Task1/NER_test.json', 'r') as f:
    task1_test_data = json.load(f)

with open('Json Task2/ATE_train.json', 'r') as f:
    task2_train_data = json.load(f)

with open('Json Task2/ATE_val.json', 'r') as f:
    task2_val_data = json.load(f)

with open('Json Task2/ATE_test.json', 'r') as f:
    task2_test_data = json.load(f)

with open('glove_embedding.pkl', 'rb') as pickle_file:
    glove_embeddings = pickle.load(pickle_file)
    
with open('fast_text_embedding.pkl', 'rb') as pickle_file:
    fast_text_embedding = pickle.load(pickle_file)

with open('word2vec_embeddings.pkl', 'rb') as pickle_file:
    word2vec_embeddings = pickle.load(pickle_file)

bio_mapping_task1 = {'B_ORG': 0, 'I_ORG': 1, 'B_RESPONDENT': 2, 'I_RESPONDENT': 3, 'B_JUDGE': 4, 'I_JUDGE': 5,
               'B_STATUTE': 6, 'I_STATUTE': 7, 'B_OTHER_PERSON': 8, 'I_OTHER_PERSON': 9, 'B_COURT': 10, 'I_COURT': 11,
               'B_GPE': 12, 'I_GPE': 13, 'B_PETITIONER': 14, 'I_PETITIONER': 15, 'B_WITNESS': 16, 'I_WITNESS': 17,
               'B_CASE_NUMBER': 18, 'I_CASE_NUMBER': 19, 'B_PRECEDENT': 20, 'I_PRECEDENT': 21, 'B_DATE': 22, 'I_DATE': 23,
               'B_PROVISION': 24, 'I_PROVISION': 25, 'O': 26}
bio_mapping_task2 = {'O' : 0, 'I' : 1, 'B' : 2}

In [5]:
def pad_data():
    max_length = 84
    pad_word = 'PAD'

    data = [task1_train_data, task1_val_data, task1_test_data, task2_train_data, task2_val_data, task2_test_data]
    for task_data in data:
        remove_keys = []
        for key in task_data:
            words = task_data[key]['text'].split(' ')
            tags = task_data[key]['labels']
            if(len(words) >= max_length):
                remove_keys.append(key)
                continue
            else:
                for _ in range(len(words), max_length):
                    words.append(pad_word)
                    tags.append('O')   
            task_data[key]['text'] = ' '.join(words)
            task_data[key]['labels'] = tags
        
        for key in remove_keys:
            task_data.pop(key, None)



pad_data()
task1_train_data = {new_key: task1_train_data[old_key] for new_key, (old_key, _) in enumerate(task1_train_data.items())}
task1_val_data = {new_key: task1_val_data[old_key] for new_key, (old_key, _) in enumerate(task1_val_data.items())}
task1_test_data = {new_key: task1_test_data[old_key] for new_key, (old_key, _) in enumerate(task1_test_data.items())}
task2_train_data = {new_key: task2_train_data[old_key] for new_key, (old_key, _) in enumerate(task2_train_data.items())}
task2_val_data = {new_key: task2_val_data[old_key] for new_key, (old_key, _) in enumerate(task2_val_data.items())}
task2_test_data = {new_key: task2_test_data[old_key] for new_key, (old_key, _) in enumerate(task2_test_data.items())}



In [12]:
class RNNModel(nn.Module):
    def __init__(self, embedding_dim, output_size):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(embedding_dim, 32, num_layers=1, batch_first=True)
        # self.fc1 = nn.Linear(128, 64)
        self.fc1 = nn.Linear(32, output_size)

    def forward(self, x):
        # x = self.embedding_layer(x)
        out, _ = self.rnn(x)
        out = self.fc1(out)
        return out
    
class LSTMModel(nn.Module):
    def __init__(self, embedding_dim, output_size):
        super(LSTMModel, self).__init__()
        # self.embedding_layer = nn.Embedding.from_pretrained(pretrained_embedding, freeze=True)
        self.lstm = nn.LSTM(embedding_dim, 32, num_layers=1, batch_first=True)
        self.fc1 = nn.Linear(32, output_size)

    def forward(self, x):
        # x = self.embedding_layer(x)
        out, _ = self.lstm(x)
        out = self.fc1(out)
        return out
    

class GRUModel(nn.Module):
    def __init__(self, embedding_dim, output_size):
        super(GRUModel, self).__init__()
        # self.embedding_layer = nn.Embedding.from_pretrained(pretrained_embedding, freeze=True)
        self.gru = nn.GRU(embedding_dim, 32, num_layers=1, batch_first=True)
        # self.fc1 = nn.Linear(128, 64)
        self.fc1 = nn.Linear(32, output_size)

    def forward(self, x):
        # x = self.embedding_layer(x)
        out, _ = self.gru(x)
        out = self.fc1(out)
        # out = self.fc2(out)
        return out
class Task_data(Dataset):
    def __init__(self, data, bio_index, embedding_type):
        self.data = data
        self.length = len(self.data)
        self.bio_index =  bio_index
        self.embedding_type = embedding_type

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        input_sequence = self.data[index]['text'].split(' ')
        sentence_embeddings = []
        if self.embedding_type == "glove":
            sentence_embeddings = [glove_embeddings.get(word, np.zeros(300, dtype = 'float32')) for word in input_sequence]
        elif self.embedding_type == "word2vec":
            sentence_embeddings = [word2vec_embeddings.get(word, np.zeros(300, dtype = 'float32')) for word in input_sequence]
        elif self.embedding_type == "fast_text":
            sentence_embeddings = [np.array(fast_text_embedding.get(word, np.zeros(300, dtype = 'float32')), dtype = 'float32') for word in input_sequence]

        sentence_embeddings = np.array(sentence_embeddings, dtype='float32')
        output_sequence = self.data[index]['labels']
        output_labels = [self.bio_index[word] for word in output_sequence]

        # output_labels = np.array(output_labels, dtype='float32')
        return torch.tensor(sentence_embeddings), torch.tensor(output_labels)

In [13]:
def test_model(task, embedding_type, model, criterion, device, batch_size = 1):
    test_dataloader = None
    if task == 1:
        test_dataloader =  DataLoader(Task_data(task1_test_data, bio_mapping_task1, embedding_type), batch_size=batch_size, shuffle=False)

    elif task == 2:
        test_dataloader =  DataLoader(Task_data(task2_test_data, bio_mapping_task2, embedding_type), batch_size=batch_size, shuffle=False)

    total_test_loss = 0
    all_test_predictions = []
    all_test_targets = []

    with torch.no_grad():
        for test_inputs, test_targets in test_dataloader:
            test_inputs, test_targets = test_inputs.to(device), test_targets.to(device)
            test_outputs = model(test_inputs)

            loss = 0
            for i in range(test_outputs.size(1)):  # Iterate over time steps
                loss += criterion(test_outputs[:, i, :], test_targets[:, i])  

            total_test_loss += loss.item()

            all_test_predictions.extend(test_outputs.argmax(dim=2).view(-1).cpu().numpy())
            all_test_targets.extend(test_targets.view(-1).cpu().numpy())

        avg_test_loss = total_test_loss / len(test_dataloader)
        test_macro_f1 = f1_score(all_test_targets, all_test_predictions, average='macro')
    print(f'Test Loss: {avg_test_loss}, Test Macro F1-Score: {test_macro_f1}')

In [14]:
criterion = nn.CrossEntropyLoss()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [15]:
task_ = 1
print("Task 1")
t1_rnn_word2vec = torch.load('trained_models/t1_rnn_word2vec.pt')
test_model(task = task_, embedding_type = "word2vec", model = t1_rnn_word2vec, criterion=criterion, device=device)
t1_rnn_glove = torch.load('trained_models/t1_rnn_glove.pt')
test_model(task = task_, embedding_type = "glove", model = t1_rnn_glove, criterion=criterion, device=device)
t1_rnn_fast_text = torch.load('trained_models/t1_rnn_fast_text.pt')
test_model(task = task_, embedding_type = "fast_text", model = t1_rnn_fast_text, criterion=criterion, device=device)

task_ = 2
print("Task 2")
t2_rnn_word2vec = torch.load('trained_models/t2_rnn_word2vec.pt')
test_model(task = task_, embedding_type = "word2vec", model = t2_rnn_word2vec, criterion=criterion, device=device)
t2_rnn_glove  = torch.load('trained_models/t2_rnn_glove.pt')
test_model(task = task_, embedding_type = "glove", model = t2_rnn_glove, criterion=criterion, device=device)
t2_rnn_fast_text = torch.load('trained_models/t2_rnn_fast_text.pt')
test_model(task = task_, embedding_type = "fast_text", model = t2_rnn_fast_text, criterion=criterion, device=device)

Task 1
Test Loss: 11.740047338547365, Test Macro F1-Score: 0.388980559944608
Test Loss: 10.376716428062391, Test Macro F1-Score: 0.41710771184468665
Test Loss: 9.752915718593716, Test Macro F1-Score: 0.3948508353035882
Task 2
Test Loss: 6.679583282732382, Test Macro F1-Score: 0.49412518400934524
Test Loss: 5.92492855176693, Test Macro F1-Score: 0.49614033381035155
Test Loss: 6.40417336618028, Test Macro F1-Score: 0.38904428761369086


In [16]:
task_ = 1
print("Task 1")
task1_lstm_word2vec = torch.load('trained_models/t1_lstm_word2vec.pt')
test_model(task = task_, embedding_type = "word2vec", model = task1_lstm_word2vec, criterion=criterion, device=device)
task1_lstm_glove = torch.load('trained_models/t1_lstm_glove.pt')
test_model(task = task_, embedding_type = "glove", model = task1_lstm_glove, criterion=criterion, device=device)
task1_lstm_fast_text = torch.load('trained_models/t1_lstm_fast_text.pt')
test_model(task = task_, embedding_type = "fast_text", model = task1_lstm_fast_text, criterion=criterion, device=device)

task_ = 2
print("Task 2")
task2_lstm_word2vec = torch.load('trained_models/t2_lstm_word2vec.pt')
test_model(task = task_, embedding_type = "word2vec", model = task2_lstm_word2vec, criterion=criterion, device=device)
task2_lstm_glove = torch.load('trained_models/t2_lstm_glove.pt')
test_model(task = task_, embedding_type = "glove", model = task2_lstm_glove, criterion=criterion, device=device)
task2_lstm_fast_text = torch.load('trained_models/t2_lstm_fast_text.pt')
test_model(task = task_, embedding_type = "fast_text", model = task2_lstm_fast_text, criterion=criterion, device=device)

Task 1
Test Loss: 12.965984447801464, Test Macro F1-Score: 0.44395642899718474
Test Loss: 11.591764899088785, Test Macro F1-Score: 0.48691259136665765
Test Loss: 8.583206160344139, Test Macro F1-Score: 0.45014480664312484
Task 2
Test Loss: 4.085268241058036, Test Macro F1-Score: 0.6393910482986204
Test Loss: 3.83084511884102, Test Macro F1-Score: 0.5835704061603947
Test Loss: 5.099730147094261, Test Macro F1-Score: 0.39782553143656035


In [17]:
task_ = 1
print("Task 1")
task1_gru_word2vec = torch.load('trained_models/t1_gru_word2vec.pt')
test_model(task = task_, embedding_type = "word2vec", model = task1_gru_word2vec, criterion=criterion, device=device)
task1_gru_glove = torch.load('trained_models/t1_gru_glove.pt')
test_model(task = task_, embedding_type = "glove", model = task1_gru_glove, criterion=criterion, device=device)
task1_gru_fast_text = torch.load('trained_models/t1_gru_fast_text.pt')
test_model(task = task_, embedding_type = "fast_text", model = task1_gru_fast_text, criterion=criterion, device=device)

task_ = 2
print("Task 2")
task2_gru_word2vec = torch.load('trained_models/t2_gru_word2vec.pt')
test_model(task = task_, embedding_type = "word2vec", model = task2_gru_word2vec, criterion=criterion, device=device)
task2_gru_glove = torch.load('trained_models/t2_gru_glove.pt')
test_model(task = task_, embedding_type = "glove", model = task2_gru_glove, criterion=criterion, device=device)
task2_gru_fast_text = torch.load('trained_models/t2_gru_fast_text.pt')
test_model(task = task_, embedding_type = "fast_text", model = task2_gru_fast_text, criterion=criterion, device=device)

Task 1
Test Loss: 11.772546144458744, Test Macro F1-Score: 0.44316043606860306
Test Loss: 10.935374668804407, Test Macro F1-Score: 0.4732700735687208
Test Loss: 8.591720511924, Test Macro F1-Score: 0.4751370632763503
Task 2
Test Loss: 4.1824229497735095, Test Macro F1-Score: 0.6403280405564583
Test Loss: 5.716632490114468, Test Macro F1-Score: 0.46937512168078505
Test Loss: 3.9069089361625475, Test Macro F1-Score: 0.6614762842504465
